In [2]:
import numpy as np
import pyvista as pv
from scipy.ndimage import map_coordinates
import matplotlib.pyplot as plt
import numpy as np
import os
from pathlib import Path
import matplotlib.pyplot as plt
from tifffile import imread
# python batch_slicer.py
#import png_to_obj
import multiprocessing
from tqdm import tqdm # 1. tqdm 라이브러리를 임포트합니다.

# --- Import the functions from our other script ---
# Make sure virtual_slicer.py is in the same directory.
try:
    from virtual_slicer import create_cube, slice_matrix, visualize_volumes, show_slice_montage
except ImportError:
    print("Error: Could not import from 'virtual_slicer.py'.")
    print("Please make sure 'virtual_slicer.py' is in the same directory as this script.")
    exit()




In [3]:
V = imread('/data/jhahn/data/brain_lightsheet/stack.tif')

In [27]:
normal_vector = [1.0, 0.0, 0.0]
normal = np.array(normal_vector) / np.linalg.norm(normal_vector)

In [28]:
original_shape = np.array(V.shape)
array_center = original_shape / 2


In [29]:
V.shape

(379, 1150, 1559)

In [30]:
int(original_shape[0]/2)

189

In [31]:
print(np.min(np.min(V[int(original_shape[0]/2)],axis=0),axis=0),np.max(np.max(V[int(original_shape[0]/2)],axis=0),axis=0))

0 255


In [32]:
# Create an orthonormal basis for the slicing plane
u = np.array([-normal[1], normal[0], 0])


if np.linalg.norm(u) < 1e-6:
    u = np.array([0, -normal[2], normal[1]])

u /= np.linalg.norm(u)
v = np.cross(normal, u)
max_dim = int(np.ceil(np.linalg.norm(original_shape)))
slice_size = (max_dim, max_dim)
slice_size

(1974, 1974)

In [33]:
# Create grid points for the new slice
x_slice, y_slice = np.meshgrid(np.arange(-max_dim/2, max_dim/2),
                                np.arange(-max_dim/2, max_dim/2))
x_slice

print(np.min(np.min(x_slice,axis=0),axis=0),np.max(np.max(x_slice,axis=0),axis=0))

-987.0 986.0


In [34]:
sliced_matrix_list = []

# Iterate through slices along the normal vector
max_travel = int(np.ceil(np.linalg.norm(array_center))) * 2
max_travel

1974

In [35]:
points_3d.shape

print(np.min(np.min(points_3d,axis=1),axis=1))
print(np.max(np.max(points_3d,axis=1),axis=1))


[-348.47828873   36.31460449  535.37968655]
[1852.11726764 2236.91016086 2146.32744072]


In [40]:
print(array_center)
print(normal)

[189.5 575.  779.5]
[1. 0. 0.]


In [41]:
s = 0
print(array_center)
print(array_center + normal * (s - max_travel / 2))

[189.5 575.  779.5]
[-797.5  575.   779.5]


In [36]:
sliced_matrix_list = []
s = 1
slice_gap = 20
slice_id = 0
for s in range(0,max_travel,slice_gap):
    pt = array_center + normal * (s - max_travel / 2)
    
    
    points_3d = pt[:, np.newaxis, np.newaxis] + \
                u[:, np.newaxis, np.newaxis] * x_slice + \
                v[:, np.newaxis, np.newaxis] * y_slice
    #print(np.min(np.min(points_3d,axis=0),axis=0),np.max(np.max(points_3d,axis=0),axis=0))

    # Sample the volume at these 3D coordinates
    # Note: map_coordinates expects coordinates in (z, y, x) order for a (depth, height, width) array
    coords_for_map = [points_3d[2], points_3d[1], points_3d[0]]
    oblique_slice = map_coordinates(V, coords_for_map, order=0, mode='constant', cval=0.0)
    if not np.any(oblique_slice):
        continue
    sliced_matrix_list.append(oblique_slice)
    print(f'{slice_id}')
    print(np.min(np.min(points_3d,axis=1),axis=1))
    print(np.max(np.max(points_3d,axis=1),axis=1))
    slice_id += 1
    #print(oblique_slice)
    print(np.min(np.min(oblique_slice,axis=0),axis=0),np.max(np.max(oblique_slice,axis=0),axis=0))

    #unique_values, counts = np.unique(oblique_slice.flatten(), return_counts=True)
    #for i in range(len(unique_values)):
    #    print(f"{unique_values[i]} : {counts[i]} ")

sliced_matrix = np.stack(sliced_matrix_list, axis=-1)


def remove_zero_slices(matrix):
    """Removes slices from a 3D matrix that contain only zeros."""
    non_zero_slices = [i for i in range(matrix.shape[2]) if np.any(matrix[:, :, i])]
    return matrix[:, :, non_zero_slices]


final_arr =  remove_zero_slices(sliced_matrix)

0
[ 202.5 -412.  -207.5]
[ 202.5 1561.  1765.5]
0 128
1
[ 222.5 -412.  -207.5]
[ 222.5 1561.  1765.5]
0 128
2
[ 242.5 -412.  -207.5]
[ 242.5 1561.  1765.5]
0 128
3
[ 262.5 -412.  -207.5]
[ 262.5 1561.  1765.5]
0 128
4
[ 282.5 -412.  -207.5]
[ 282.5 1561.  1765.5]
0 128
5
[ 302.5 -412.  -207.5]
[ 302.5 1561.  1765.5]
0 255
6
[ 322.5 -412.  -207.5]
[ 322.5 1561.  1765.5]
0 255
7
[ 342.5 -412.  -207.5]
[ 342.5 1561.  1765.5]
0 255
8
[ 362.5 -412.  -207.5]
[ 362.5 1561.  1765.5]
0 255
9
[ 382.5 -412.  -207.5]
[ 382.5 1561.  1765.5]
0 255
10
[ 402.5 -412.  -207.5]
[ 402.5 1561.  1765.5]
0 255
11
[ 422.5 -412.  -207.5]
[ 422.5 1561.  1765.5]
0 255
12
[ 442.5 -412.  -207.5]
[ 442.5 1561.  1765.5]
0 255
13
[ 462.5 -412.  -207.5]
[ 462.5 1561.  1765.5]
0 255
14
[ 482.5 -412.  -207.5]
[ 482.5 1561.  1765.5]
0 255
15
[ 502.5 -412.  -207.5]
[ 502.5 1561.  1765.5]
0 255
16
[ 522.5 -412.  -207.5]
[ 522.5 1561.  1765.5]
0 255
17
[ 542.5 -412.  -207.5]
[ 542.5 1561.  1765.5]
0 255
18
[ 562.5 -412.  -2

In [64]:
unique_values, counts = np.unique(final_arr.flatten(), return_counts=True)
print(unique_values, counts)

_final_arr = final_arr.copy().astype(float)
_final_arr[_final_arr==128] = 0.5
_final_arr[_final_arr==255] = 1.0
unique_values, counts = np.unique(_final_arr.flatten(), return_counts=True)
print(unique_values, counts)
#for i in range(len(unique_values)):
#    print(f"{unique_values[i]} : {counts[i]} ")


[  0 128 255] [182219361   8291567    426196]
[0.  0.5 1. ] [182219361   8291567    426196]


In [63]:
_final_arr

array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       ...,

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 

In [ ]:
slice_gap = 20
num_print = 0
for s in range(0,max_travel,slice_gap):
    
    # Calculate the center point for the current slice
    pt = array_center + normal * (s - max_travel / 2)
    #pt = array_center + normal * (s - max_travel / 2)
    # Check if the plane is reasonably within the volume bounds
    '''
    if not (0 <= pt[0] < original_shape[0] and 
            0 <= pt[1] < original_shape[1] and 
            0 <= pt[2] < original_shape[2]):
        if len(sliced_matrix_list) > 0 : # Stop if we have collected slices and moved out
            print('*'*10)
            print(pt,original_shape)
            break
        else: # continue until we are in the volume
            print('@@'*10)
            print(pt,original_shape)
            continue
    '''
    num_print += 1

    # Convert 2D slice grid points to 3D coordinates in the original volume
    points_3d = pt[:, np.newaxis, np.newaxis] + \
                u[:, np.newaxis, np.newaxis] * x_slice + \
                v[:, np.newaxis, np.newaxis] * y_slice
    # Sample the volume at these 3D coordinates
    # Note: map_coordinates expects coordinates in (z, y, x) order for a (depth, height, width) array
    coords_for_map = [points_3d[2], points_3d[1], points_3d[0]]
    oblique_slice = map_coordinates(V, coords_for_map, order=0, mode='constant', cval=0.0)
    

    # If the slice is not empty, add it to our list
    if np.any(oblique_slice):
        sliced_matrix_list.append(oblique_slice)

if not sliced_matrix_list:
    print("Warning: No data was captured in the slices. The volume might be empty or the normal is pointing away.")
    return np.zeros((10, 10, 10)) # Return a small empty matrix

# Stack the collected slices into a new 3D matrix
sliced_matrix = np.stack(sliced_matrix_list, axis=-1)
print('sliced_matrix',sliced_matrix.shape)